# Ray RLlib Multi-Armed Bandits - Linear Thompson Sampling

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This lesson uses a second exploration strategy we discussed briefly in lesson [02 Exploration vs. Exploitation Strategies](02-Exploration-vs-Exploitation-Strategies.ipynb), _Thompson Sampling_, with a linear variant, [LinTS](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints).

## Wheel Bandit

We'll use it on the `Wheel Bandit` problem ([RLlib discrete.py source code](https://github.com/ray-project/ray/blob/master/rllib/contrib/bandits/envs/discrete.py)), which is an artificial problem designed to force exploration. It is described in the paper [Deep Bayesian Bandits Showdown](https://arxiv.org/abs/1802.09127) (see _The Wheel Bandit_ section). The paper uses it to  model 2D contexts, but it can be generalized to more than two dimensions.

You can visualize this problem as a wheel (circle) with four other regions around it. An exploration parameter delta $\delta$ defines a threshold, such that if the norm of the context vector is less than or equal to delta (inside the “wheel”) then the leader action is taken (conventionally numbered `1`). Otherwise, the other four actions are explored.

From figure 3 in [Deep Bayesian Bandits Showdown](https://arxiv.org/abs/1802.09127), the Wheel Bandit can be visualized this way:

![Wheel Bandit](../../images/rllib/Wheel-Bandit.png)

The radius of the entire colored circle is 1.0, while the radius of the blue "core" is $\delta$.

Contexts are sampled randomly within the unit circle (radius 1.0). The optimal action for the blue, red, green, black, or yellow region is the action 1, 2, 3, 4, or 5, respectively. In other words, if the context is in the blue region, radius < $\delta$, action 1 is optimal, if it is in the upper-right-hand quadrant with radius between $\delta$ and 1.0, then action 2 is optimal, etc.

The parameter $\delta$ controls how aggressively we explore. The reward $r$ for each action and context combination are based on a normal distribution as follows:

Action 1 offers the reward, $r \sim \mathcal{N}({\mu_1,\sigma^2})$, independent of context.

Actions 2-5 offer the reward, $r \sim \mathcal{N}({\mu_2,\sigma^2})$ where $\mu_2 < \mu_1$, _when they are suboptimal choices_. When they are optimal, the reward is $r \sim \mathcal{N}({\mu_3,\sigma^2})$ where $\mu_3 \gg \mu_1$.

In addition to $\delta$, the parameters $\mu_1$, $\mu_2$ $\mu_3$, and $\sigma$ are configurable. The default values for these parameters in the paper and in the [RLlib implementation](https://github.com/ray-project/ray/blob/master/rllib/contrib/bandits/envs/discrete.py) are as follows:

```python
DEFAULT_CONFIG_WHEEL = {
    "delta": 0.5,
    "mu_1": 1.2,
    "mu_2": 1.0,
    "mu_3": 50.0,
    "std": 0.01  # sigma
}
```

Note that the probability of a context randomly falling in the high-reward region (not blue) is 1 − $\delta^2$. Therefore, the difficulty of the problem increases with $\delta$, and algorithms used with this bandit are more likely to get stuck repeatedly selecting action 1 for large $\delta$.

## Use Wheel Bandit with Thompson Sampling

Note the import in the next cell of `LinTSTrainer` and how it is used below when setting up the _Tune_ job. For the `LinUCB` example in the [previous lesson](04-Linear-Upper-Confidence-bound.ipynb), we didn't import the corresponding `LinUCBTrainer`, but passed a "magic" string to Tune, `contrib/LinUCB`, which RLlib already knows how to associate with the corresponding `LinUCBTrainer` implementation. Passing the class explicitly, as we do here, is an alternative. The [RLlib environments documentation](https://docs.ray.io/en/latest/rllib-env.html) discusses these techniques.

In [ ]:
import time
import numpy as np
import pandas as pd
import ray
from ray.rllib.contrib.bandits.agents import LinTSTrainer
from ray.rllib.contrib.bandits.agents.lin_ts import TS_CONFIG
from ray.rllib.contrib.bandits.envs import WheelBanditEnv

In [ ]:
from bokeh_util import plot_cumulative_regret, plot_wheel_bandit_model_weights
# The next two lines prevent Bokeh from opening the graph in a new window.
import bokeh
bokeh.io.reset_output()
bokeh.io.output_notebook()

In [ ]:
wbe = WheelBanditEnv()
wbe.config

The effective number of `training_iterations` will be `20 * timesteps_per_iteration == 2,000` where the timesteps per iteration is `100` by default.

In [ ]:
TS_CONFIG["env"] = WheelBanditEnv

training_iterations = 20
print("Running training for %s time steps" % training_iterations)

What's in the standard config object for _LinTS_ anyway??

In [ ]:
TS_CONFIG

Initialize Ray...

In [ ]:
!../../tools/start-ray.sh --check --verbose

In [ ]:
ray.init(address='auto', ignore_reinit_error=True)

In [ ]:
analysis = ray.tune.run(
    LinTSTrainer,
    config=TS_CONFIG,
    stop={"training_iteration": training_iterations},
    num_samples=2,
    checkpoint_at_end=True,
    verbose=1
)

How long did it take?

In [ ]:
stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

Analyze cumulative regrets of the trials

In [ ]:
df = pd.DataFrame()

for key, df_trial in analysis.trial_dataframes.items():
    df = df.append(df_trial, ignore_index=True)

regrets = df \
    .groupby("info/num_steps_trained")["info/learner/default_policy/cumulative_regret"] \
    .aggregate(["mean", "max", "min", "std"])

In [ ]:
regrets

In [ ]:
plot_cumulative_regret(regrets)

As always, here is an [image](../../images/rllib/LinTS-Cumulative-Regret-05.png) from a previous run. How similar is your graph? We have observed a great deal of variability from one run to the next, more than we have seen with _LinUCB_. This suggests that extra caution is required when using _LinTS_ to ensure that good results are achieved.

Here is how you can restore a trainer from a checkpoint:

In [ ]:
trial = analysis.trials[0]
trainer = LinTSTrainer(config=TS_CONFIG)
trainer.restore(trial.checkpoint.value)

Get model to plot arm weights distribution

In [ ]:
model = trainer.get_policy().model
means = [model.arms[i].theta.numpy() for i in range(5)]
covs = [model.arms[i].covariance.numpy() for i in range(5)]
model, means, covs

Plot weight distributions for different arms

In [ ]:
plot_wheel_bandit_model_weights(means, covs)

Here is an [image](../../images/rllib/LinTS-Weight-Distribution-of-Arms-05.png) from a previous run. How similar is your graph?

## Exercise 1

Experiment with different $\delta$ values, for example 0.7 and 0.9. What do the cumulative regret and weights graphs look like? 

You can set the $\delta$ value like this:

```python
TS_CONFIG["delta"] = 0.7
```

See the [solutions notebook](solutions/Multi-Armed-Bandits-Solutions.ipynb) for discussion of this exercise.

In [ ]:
ray.shutdown()  # "Undo ray.init()".